# 0. Введение

В современном мире анализ данных и машинное обучение играют ключевую роль в решении широкого спектра задач, включая прогнозирование и классификацию. В этом контексте, набор данных "Vinho Verde", состоящий из двух поднаборов, связанных с португальскими вариантами вина "Vinho Verde", предоставляет уникальную возможность исследования и применения методов машинного обучения для классификации.

Этот набор данных был собран с целью анализа физико-химических параметров (входные данные) и сенсорных характеристик (выходные данные) различных вариантов этого вина. Особенностью данного набора данных является отсутствие информации о многих нетехнических аспектах, таких как сорт винограда, бренд вина и его цена. Это создает интересные вызовы и возможности для анализа данных и построения моделей.

В данном исследовании мы сфокусируемся на решении задачи бинарной классификации с использованием данных из набора "Vinho Verde". Наша цель - разработать модель, способную классифицировать вина на два класса на основе физико-химических и сенсорных характеристик. Помимо этого, набор данных предоставляет нам вызовы, связанные с несбалансированными классами, что требует особого внимания при разработке модели.

# 1. Загрузка библиотек

In [1]:
from scripts import save_model
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report

# 2. Загрузка данных

In [2]:
df = pd.read_csv('data/winequalityN.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


# 3. Предобработка

In [3]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6487.000000,6489.000000,6494.000000,6495.000000,6495.000000,6497.000000,6497.000000,6497.000000,6488.000000,6493.000000,6497.000000,6497.000000
mean,7.216579,0.339691,0.318722,5.444326,0.056042,30.525319,115.744574,0.994697,3.218395,0.531215,10.491801,5.818378
std,1.296750,0.164649,0.145265,4.758125,0.035036,17.749400,56.521855,0.002999,0.160748,0.148814,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [4]:
df.dropna(inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6463 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6463 non-null   object 
 1   fixed acidity         6463 non-null   float64
 2   volatile acidity      6463 non-null   float64
 3   citric acid           6463 non-null   float64
 4   residual sugar        6463 non-null   float64
 5   chlorides             6463 non-null   float64
 6   free sulfur dioxide   6463 non-null   float64
 7   total sulfur dioxide  6463 non-null   float64
 8   density               6463 non-null   float64
 9   pH                    6463 non-null   float64
 10  sulphates             6463 non-null   float64
 11  alcohol               6463 non-null   float64
 12  quality               6463 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 706.9+ KB


In [6]:
df['is_red'] = df.type.apply(lambda x: 1 if x == 'red' else 0)
df.drop('type', inplace=True, axis=1)
df.tail()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,is_red
6491,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,1
6492,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1
6494,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1
6495,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,1
6496,6.0,0.310,0.47,3.6,0.067,18.0,42.0,0.99549,3.39,0.66,11.0,6,1


In [7]:
df.to_csv('data/wine_for_train.csv')

# 4. Подготовка датасета для обучения

In [8]:
X = df.drop('quality', axis=1)
y = df.quality.apply(lambda x: 1 if x > 5 else 0)
X.shape, y.shape

((6463, 12), (6463,))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test.shape, y_test.shape

((1293, 12), (1293,))

# 5. Обучение моделей

In [10]:
log_r = LogisticRegression(max_iter=10000, random_state=42)
log_r.fit(X_train, y_train)
print(classification_report(y_test, log_r.predict(X_test)))

              precision    recall  f1-score   support

           0       0.71      0.56      0.62       482
           1       0.77      0.86      0.81       811

    accuracy                           0.75      1293
   macro avg       0.74      0.71      0.72      1293
weighted avg       0.74      0.75      0.74      1293



In [11]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
print(classification_report(y_test, rf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.79      0.72      0.75       482
           1       0.84      0.89      0.86       811

    accuracy                           0.82      1293
   macro avg       0.82      0.80      0.81      1293
weighted avg       0.82      0.82      0.82      1293



In [12]:
gbc = GradientBoostingClassifier(random_state=42)
gbc.fit(X_train, y_train)
print(classification_report(y_test, gbc.predict(X_test)))

              precision    recall  f1-score   support

           0       0.73      0.63      0.68       482
           1       0.80      0.86      0.83       811

    accuracy                           0.78      1293
   macro avg       0.77      0.75      0.75      1293
weighted avg       0.77      0.78      0.77      1293



In [13]:
models = {'LogisticRegression' :log_r, 'RandomForestClassifier' :rf, 'GradientBoostingClassifier' :gbc}

for model in models.keys():
    save_model.save_model(models[model], model)